In [1]:
import numpy as np
import pandas as pd
import cv2
import os
import PIL

import pathlib
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D,Dense,Dropout, Flatten,Activation, BatchNormalization,MaxPooling2D
from tensorflow.keras import datasets, layers, models
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array

In [ ]:
!unzip /content/drive/MyDrive/DL_CNN.zip

In [3]:
data_dir = ("/content/DL_CNN")
print(data_dir)

/content/DL_CNN


In [4]:
data_dir = pathlib.Path("/content/DL_CNN")
print(data_dir)
list(data_dir.glob('*/*.jpg'))[:5]

/content/DL_CNN


[PosixPath('/content/DL_CNN/Dog/4178.jpg'),
 PosixPath('/content/DL_CNN/Dog/7539.jpg'),
 PosixPath('/content/DL_CNN/Dog/1734.jpg'),
 PosixPath('/content/DL_CNN/Dog/11634.jpg'),
 PosixPath('/content/DL_CNN/Dog/2842.jpg')]

In [5]:
dogs = list(data_dir.glob('Dog/*'))
dogs[:5]

[PosixPath('/content/DL_CNN/Dog/4178.jpg'),
 PosixPath('/content/DL_CNN/Dog/7539.jpg'),
 PosixPath('/content/DL_CNN/Dog/1734.jpg'),
 PosixPath('/content/DL_CNN/Dog/11634.jpg'),
 PosixPath('/content/DL_CNN/Dog/2842.jpg')]

In [6]:
cats = list(data_dir.glob('Cat/*'))
cats[:5]

[PosixPath('/content/DL_CNN/Cat/165.jpg'),
 PosixPath('/content/DL_CNN/Cat/2970.jpg'),
 PosixPath('/content/DL_CNN/Cat/4178.jpg'),
 PosixPath('/content/DL_CNN/Cat/8286.jpg'),
 PosixPath('/content/DL_CNN/Cat/7539.jpg')]

In [7]:
image_count = len(list(data_dir.glob('*/*.jpg')))
print(image_count)

13638


In [8]:
X, Y = [], []

In [ ]:
PIL.Image.open(str(dogs[1]))

In [10]:
pet_images_dict = {
    'cats': list(data_dir.glob('Cat/*')),
    'dogs': list(data_dir.glob('Dog/*')),
}

pet_labels_dict = {
    'cats': 0,
    'dogs': 1,
}

In [35]:
IMAGE_WIDTH=100
IMAGE_HEIGHT=100
X, Y = [], []

for pet_name, images in pet_images_dict.items():
    print(pet_name)
    for image in images:
        img = cv2.imread(str(image))
        if isinstance(img,type(None)): 
            #print('image not found')
            continue
            
        elif ((img.shape[0] >= IMAGE_HEIGHT) and  (img.shape[1] >=IMAGE_WIDTH)):
            resized_img = cv2.resize(img,(IMAGE_WIDTH,IMAGE_HEIGHT))
            X.append(resized_img)
            Y.append(pet_labels_dict[pet_name])
        else:
            #print("Invalid Image")
            continue

cats
dogs


In [36]:
X = np.array(X)
Y = np.array(Y)

In [37]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, random_state=0)
print(len(X_train),len(Y_train))
print(len(X_test),len(Y_test))
print(X_train.shape)
print(Y_train.shape)
print(X_test.shape) 
print(Y_test.shape)

10140 10140
3381 3381
(10140, 100, 100, 3)
(10140,)
(3381, 100, 100, 3)
(3381,)


In [14]:
len(Y_test)

3286

In [38]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(4, (3, 3), activation='relu',input_shape=(100, 100, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(4, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(1, activation='sigmoid'))

In [39]:
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_7 (Conv2D)           (None, 98, 98, 4)         112       
                                                                 
 max_pooling2d_8 (MaxPooling  (None, 49, 49, 4)        0         
 2D)                                                             
                                                                 
 conv2d_8 (Conv2D)           (None, 47, 47, 4)         148       
                                                                 
 max_pooling2d_9 (MaxPooling  (None, 23, 23, 4)        0         
 2D)                                                             
                                                                 
 max_pooling2d_10 (MaxPoolin  (None, 11, 11, 4)        0         
 g2D)                                                            
                                                      

In [40]:
mkdir CNN-Model

mkdir: cannot create directory ‘CNN-Model’: File exists


In [41]:
backup_dir = "/content/CNN-Model"

In [42]:
import tensorflow as tf
call_backs = [tf.keras.callbacks.TensorBoard(log_dir='./log'),
              tf.keras.callbacks.BackupAndRestore(backup_dir),
              tf.keras.callbacks.ModelCheckpoint("/content/CNN-Model",monitor="val_loss",save_best_only=True,save_weights_only=True,mode="auto",save_freq="epoch",
    
)]

In [43]:
model.compile(optimizer="adam",
              loss=tf.keras.losses.BinaryCrossentropy(),metrics=['accuracy'])
history = model.fit(X_train, Y_train, epochs=15,validation_data=(X_test, Y_test),callbacks = call_backs)

Epoch 1/15
317/317 [==============================] - 3s 7ms/step - loss: 1.5595 - accuracy: 0.7760 - val_loss: 0.6023 - val_accuracy: 0.8128
Epoch 2/15
317/317 [==============================] - 2s 6ms/step - loss: 0.4992 - accuracy: 0.8735 - val_loss: 0.4450 - val_accuracy: 0.8991
Epoch 3/15
317/317 [==============================] - 2s 6ms/step - loss: 0.4072 - accuracy: 0.9084 - val_loss: 0.3974 - val_accuracy: 0.9130
Epoch 4/15
317/317 [==============================] - 2s 6ms/step - loss: 0.3630 - accuracy: 0.9141 - val_loss: 0.3649 - val_accuracy: 0.9160
Epoch 5/15
317/317 [==============================] - 2s 6ms/step - loss: 0.3384 - accuracy: 0.9157 - val_loss: 0.3381 - val_accuracy: 0.9184
Epoch 6/15
317/317 [==============================] - 2s 6ms/step - loss: 0.3225 - accuracy: 0.9158 - val_loss: 0.3235 - val_accuracy: 0.9181
Epoch 7/15
317/317 [==============================] - 2s 6ms/step - loss: 0.3104 - accuracy: 0.9159 - val_loss: 0.3156 - val_accuracy: 0.9181
Epoch 

In [44]:
model.save('cnn_catvsdog.h5')

In [45]:
test_loss, test_acc = model.evaluate(X_test,  Y_test)

106/106 [==============================] - 0s 3ms/step - loss: 0.2824 - accuracy: 0.9187


In [46]:
test_acc*100

91.86631441116333

In [24]:
x = model.predict(X_test)

In [ ]:
x

array([[1.2786014e-01],
       [7.4671054e-01],
       [2.5316641e-07],
       ...,
       [6.4983493e-01],
       [2.6352942e-02],
       [3.5700354e-01]], dtype=float32)

In [ ]:
model.predict(X_test[:2])

array([[0.12786005],
       [0.7467105 ]], dtype=float32)

In [ ]:
print(x[10])

[0.19007336]


In [ ]:
print(x.shape)

(3286, 1)


In [ ]:


x[0]

array([0.12786014], dtype=float32)

In [47]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [48]:

import tensorflow as tf
import datetime

In [ ]:
%tensorboard --logdir log/fit

In [50]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6007: logdir log/fit (started 0:00:00 ago; pid 729)
  - port 6006: logdir reduced7/fit (started 0:04:15 ago; pid 514)


In [ ]:
# Control TensorBoard display. If no port is provided, 
# the most recently launched TensorBoard is used
notebook.display(port=6006, height=1000)

In [52]:
!tensorboard dev upload \
  --logdir log \
  --name "CATVSDOG_CNN" \
  --description "TB" \
  --one_shot


New experiment created. View your TensorBoard at: https://tensorboard.dev/experiment/J16nPeOsQxG3iK6xYdqbqg/

[2022-09-09T06:13:22] Started scanning logdir.
[2022-09-09T06:13:23] Total uploaded: 90 scalars, 0 tensors, 1 binary objects (47.3 kB)
[2022-09-09T06:13:23] Done scanning logdir.


Done. View your TensorBoard at https://tensorboard.dev/experiment/J16nPeOsQxG3iK6xYdqbqg/


In [ ]:
cat_dir = ("/content/drive/MyDrive/CAT")

In [ ]:
cat_dir = pathlib.Path("/content/drive/MyDrive/CAT")
cat_dir

PosixPath('/content/drive/MyDrive/CAT')

In [ ]:
 d = list(cat_dir.glob('*.jpg'))

In [ ]:
dict = { 'c' : list(cat_dir.glob('*.jpg'))}


In [ ]:
X_t = np.array(d)

In [ ]:
x = model.predict(X_t)

ValueError: ignored

In [ ]:
from PIL import Image
img = Image.fromarray(X_test[10],"RGB")
img.save("test_img.png")
img.show()

In [ ]:
PIL.Image.open(str("test_img.png"))